In [1]:
# main.ipynb
# Archivo base en el que se maneja la experimentación del diseño metodológico.
# Objetivo general: Utilizar los datos publicos (ubicados en la carpeta /data/)
#     de municipios en los que existe información para diseñar una estrategia
#     para distribuir alimentos en el territorio nacional.
# Objetivos específicos:

#     ---pronostico_poblacional---
#     [IA]1. Determinar por municipio que herramienta es mejor para pronosticar
#            la población en 10 años a partir del año actual.
#               a. Regresión Lineal Multiple || Multiple Linear Regression.
#               b. Arboles de Regresión || Regression Tree.
#               c. Máquinas de Vectores de Soporte || Support Vector Machine.
#               d. Bosques Aleatorios || Random Forest Regression.
#               e. Redes neurales || Deep Learning.
#               f. Regresión Tradicional.
#     ---capacidad_y_costo---
#     [IO]2. Determinar las capacidades y costos de almacenamiento de alimentos
#            en cada municipio utilizando los datos del pronóstico, los datos
#            abiertos y consideraciones de los analístas.
#     ---pronosticar_capacidad_y_costo---
#     [IA]3. Entrenar un clasificador desde los datos de capacidades y costos
#            de almacenamiento por municipio.
#               a. Arboles de Decision || Decision Tree.
#               b. Análisis Discriminante Lineal || Linear Discriminant Analysis.
#               c. Regresión Logística || Logistic Regression.
#               d. Máquinas de Vectores de Soporte || Support Vector Machine.
#               e. Redes Neurales || Deep Learning.
#               f. Análisis de frecuencias. [Aún está en desarrollo la idea]
#     ---cantidad_de_clusteres---
#     [IO]4. Proponer una cantidad de clusteres que permitan disminuir el costo
#            computacional del algoritmo optimizador y compararlo con los métodos
#            tradicionales.
#     ---generar_clusteres---
#     [IA]5. Dividir los municipios en clústeres, rectificando que es viable
#            satisfacer la demanda de alimentos con la capacidad instalada del
#            municipio.
#               a. k-means.
#               b. Mapa Autoorganizado || Self-Organizing Map.
#               c. Agrupamiento Jerárquico || Agglomerative Clustering.
#               d. DBSCAN.
#     ---solucionar_cflp---
#     [IO]6. Resolver el cflp para diferentes escenarios.
#               a. Solución ingenua (todos los municipios tienen un centro de
#                  distribución).
#               b. Datos completos sin clusterizar.
#               c. Dividido por clústers.
#


In [2]:
# Manejo básico de archivos
import os, sys, warnings

# Registro del tiempo
import time

# Manejo de datos
import pandas as pd
import numpy as np

# Funciones personalizadas de /funciones/funciones.py, /funciones/alistamiento.py, /funciones/miscelaneas.py
from funciones.funciones import *
from funciones.alistamiento import *
from funciones.miscelaneas import *

from funciones.pronostico_poblacional import *

# Funciones de Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos de Machine Learning para pronóstico
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# Métricas de evaluación
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# Manejo de las advertencias
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter(action="ignore", category=ConvergenceWarning)


In [3]:
# Funciones


def seleccionar_datos_historicos(datos: pd.DataFrame, años: range) -> pd.DataFrame:
    """
    Selecciona los datos históricos de los años que se desean pronosticar.

    Parámetros:
    -----------
    datos: pd.DataFrame
        Datos de los municipios.
    años: range
        Años que se desean pronosticar.

    Retorna:
    --------
    pd.DataFrame
        Datos históricos de los años que se desean pronosticar.
    """
    datos = datos[[str(x) for x in años]]
    datos.columns = datos.columns.astype(int)
    datos = datos.replace(0, np.nan)
    return datos


def definicion_de_modelos_de_regresion(RANDOM_SEED):
    # modelos = {
    #     "Regresión Lineal Múltiple": LinearRegression(),
    #     "Árboles de Regresión": DecisionTreeRegressor(random_state=RANDOM_SEED),
    #     "Máquinas de Vectores de Soporte": SVR(C=1.0, kernel="rbf", gamma="scale"),
    #     "Bosques Aleatorios": RandomForestRegressor(random_state=RANDOM_SEED),
    #     "Redes Neuronales": MLPRegressor(max_iter=1000, random_state=RANDOM_SEED),
    # }
    modelos = {
        "Multiple Linear Regression": LinearRegression(),
        "Regression Tree": DecisionTreeRegressor(
            max_depth=100,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=RANDOM_SEED,
        ),
        "Support Vector Machine": SVR(C=1.0, kernel="rbf", gamma="scale"),
        "Random Forest Regression": RandomForestRegressor(
            n_estimators=10,
            max_depth=5,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=RANDOM_SEED,
        ),
        "Neural Network for population regression": MLPRegressor(
            hidden_layer_sizes=(10, 10, 10, 10, 10),
            activation="logistic",
            solver="adam",
            alpha=0.01,
            batch_size="auto",
            learning_rate="adaptive",
            learning_rate_init=0.01,
            max_iter=1000,
            shuffle=True,
            random_state=RANDOM_SEED,
        ),
    }
    return modelos


In [4]:
# PARAMETROS

from calendar import c


comida_per_capita = 0.00087617  # 0.87617 kg por persona
densidad_de_alimentos = 537 / 1000  # 537 kg por m3 (5 metros de altura)
RANDOM_SEED = 11

crear_estructura_de_archivos()
procesar_datos_completos(comida_per_capita, densidad_de_alimentos)
procesar_datos_imperfectos()

# Esquema general de la experimentación
# 1. Seleccionar los datos Historicos teniendo en cuenta su estructura
#    divipola | 1985 | ... | 2023
#    en donde los datos pueden existir o no en los años.
# 2. Por Municipio
#    2.1. Seleccionar los datos de los años que existen.
#    2.2. Dividir los datos en entrenamiento y prueba. (80% - 20%)
#    2.3. Entrenar los modelos.
#    2.4. Evaluar los modelos.
#    2.5. Guardar los resultados.
# 3. Sacar las métricas generales por modelo.
# 4. Escoger el mejor modelo.
# 5. Guardar los resultados.

datos_completos = pd.read_csv("data/datos_completos/municipios.csv", index_col=0)
datos_imperfectos = pd.read_csv("data/datos_imperfectos/municipios.csv", index_col=0)


# 1. Seleccionar los datos Historicos teniendo en cuenta su estructura
datos = datos_imperfectos

municipios = seleccionar_datos_historicos(datos, range(1985, 2024))
predicciones = municipios.copy()

scaler = StandardScaler()
municipios = pd.DataFrame(
    scaler.fit_transform(municipios), columns=municipios.columns, index=municipios.index
)

# 2. Por Municipio
progreso = 0
resultados = {
    "Modelo": [],
    "Municipio": [],
    "R2": [],
    "MAE": [],
    "MSE": [],
    "RMSE": [],
    "tiempo": [],
}
modelos = definicion_de_modelos_de_regresion(RANDOM_SEED)

for divipola, municipio in municipios.iterrows():
    municipio = municipio.dropna()
    X = municipio.index.values.reshape(-1, 1)
    y = municipio.values
    if len(y) < 5:
        print(
            f"El municipio {divipola} no tiene suficientes datos,\
               con {len(y)} años"
        )
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_SEED
    )
    for name, modelo in modelos.items():
        tiempo_inicial = time.time()
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        tiempo_final = time.time()
        resultados["Modelo"].append(name)
        resultados["Municipio"].append(divipola)
        resultados["R2"].append(r2_score(y_test, y_pred))
        resultados["MAE"].append(mean_absolute_error(y_test, y_pred))
        resultados["MSE"].append(mean_squared_error(y_test, y_pred))
        resultados["RMSE"].append(np.sqrt(mean_squared_error(y_test, y_pred)))
        resultados["tiempo"].append(tiempo_final - tiempo_inicial)
        # prediccion de la poblacion en 2034 en escalas originales
        progreso += 1

        print(
            f"Progreso: {progreso/(municipios.shape[0] * len(modelos))*100:0.2f}%",
            end="\r",
        )

# 3. Sacar las métricas generales por modelo.
#     es decir, filtrar los resultados (R2 > 0.9) y
resultados = pd.DataFrame(resultados)
resultados["mejor_modelo"] = resultados.groupby("Municipio")["R2"].transform("max")
resultados["mejor_modelo_nombre"] = resultados.groupby("Municipio")["Modelo"].transform(
    lambda x: x[x.idxmax()]
)
municipios_con_mejor_modelo = resultados.groupby("Municipio")[
    "mejor_modelo_nombre"
].first()

assert (
    municipios_con_mejor_modelo.shape[0] == municipios.shape[0]
), "No se ha seleccionado un modelo por municipio"

resultados = resultados[
    resultados["R2"] > 0.8
]  # Seleccionar los resultados con R2 mayor a 0.9

reporte_de_resultados = resultados.groupby("Modelo").agg(
    {
        "Municipio": "count",
        "mejor_modelo": ["mean", "std"],
        "mejor_modelo_nombre": "first",
        "R2": ["min", "max", "mean", "std"],
        "MAE": ["min", "max", "mean", "std"],
        "MSE": ["min", "max", "mean", "std"],
        "RMSE": ["min", "max", "mean", "std"],
        "tiempo": ["min", "max", "mean", "std"],
    }
)


display(reporte_de_resultados)


Procesando los DATOS COMPLETOS
Los datos ya fueron procesados
Procesando los DATOS IMPERFECTOS
    Cantidad de Municipios que demuestran inconsistencia en via terrestre               6
    Cantidad de valores no existentes:             40                 previo a ajuste de distancias
